In [66]:
import matplotlib as mpl
mpl.use("tkagg")
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import re

In [67]:
p = re.compile(r"df_(([a-zA-Z_]+)(\d{3})?)_(\d{2}-\d{2}_\d{2}).csv")

In [134]:
df_master = pd.DataFrame()
csv_dir = "./out/"
first_col_name = "Unnamed: 0"
xvar = "distance"
yvar = "sensitivity_student"
pxscale = 0.035
grp_num = 0
slice_after = 3
for d in os.listdir(csv_dir):
    m = p.match(d)
    if m is not None:
        df_temp = pd.read_csv(csv_dir + d)
        t = np.array(df_temp[first_col_name])[slice_after:]
        x = np.array(df_temp[xvar])[slice_after:]
        xas = pxscale * x
        y = np.array(df_temp[yvar])[slice_after:]
        ymag = -2.5*np.log10(y)
        algo = m.group(2)
        if m.group(3) is not None:
            algo += "_pca"

        df_temp_mod = pd.DataFrame({"t": t, "x": x, "xas": xas, "y": y, "ymag": ymag,
            "algo": algo, "run": m.group(4), "grp_num": grp_num})
        df_master = pd.concat((df_master, df_temp_mod))
        grp_num += 1

In [135]:
df_master["algo"].unique()

array(['npy_adi', 'adi_ann', 'sdi', 'dbl_pca', 'fd', 'sng_pca', 'ann_pca',
       'adi', 'asdi'], dtype=object)

In [136]:
# in future xvals need to correspond to each individual series - minor difference though
xvals = pxscale * np.array(pd.read_csv(csv_dir + d)[xvar])[slice_after:]
run_vals = df_master["run"].unique()
algo_vals = df_master["algo"].unique()

In [71]:
use_mag = True
if use_mag:
    df_run = df_master[["ymag", "t", "run"]]
else:
    df_run = df_master[["y", "t", "run"]]
# df_run_grp = df_run.groupby(by=["run", "t"])
df_run_means = df_run.groupby(by=["run", "t"]).mean(numeric_only=True)
df_run_stds = df_run.groupby(by=["run", "t"]).std(numeric_only=True)

In [72]:
run_name_map = {"45-74_20": "30ch 111fr", "45-74_10": "30ch 222fr", "40-79_20": "40ch 111fr"}

In [73]:
plot_pl = True
ylims = (0, 15) if use_mag else (1e-4, 2e-1)
xpl = 21.403 * pxscale
ypl = 45.6 / 16231.
figsize=(10,10)

plt.close()
fig, ax = plt.subplots(figsize=figsize)

if not use_mag:
    ax.set_yscale("log")
else:
    ax.invert_yaxis()
    ypl = -2.5*np.log10(ypl)

for run_name in run_vals:
    ymeans = np.array(df_run_means.loc[run_name]).reshape(-1)
    ystds = np.array(df_run_stds.loc[run_name]).reshape(-1)

    ybottom = ymeans-ystds
    ytop = ymeans+ystds

    ax.plot(xvals, ymeans, label=run_name_map[run_name])
    ax.fill_between(xvals, ybottom, ytop, alpha=0.15)

if plot_pl:
    ax.plot(xpl, ypl, "gs", label="HD 1160 B", markersize=16)


# ax.set_ylim(ylims)
ax.set_title("Contrast Curves grouped by Dataset", fontsize=24)
ax.set_xlabel("Distance (arcsec)", fontsize=20)
ax.set_ylabel("$\Delta$mag" if use_mag else "5$\sigma$ Contrast", fontsize=20)
ax.tick_params(labelsize=14)
ax.legend(fontsize=16)
fig.tight_layout()
plt.show()

In [74]:
use_mag = True
if use_mag:
    df_algo = df_master[["ymag", "t", "algo"]]
else:
    df_algo = df_master[["y", "t", "algo"]]
# df_run_grp = df_run.groupby(by=["run", "t"])
df_algo_means = df_algo.groupby(by=["algo", "t"]).mean(numeric_only=True)
df_algo_stds = df_algo.groupby(by=["algo", "t"]).std(numeric_only=True)

In [75]:
plot_pl = True
ylims = (0, 15) if use_mag else (1e-4, 2e-1)
xpl = 21.403 * pxscale
ypl = 45.6 / 16231.
figsize = (10, 10)

plt.close()
fig, ax = plt.subplots(figsize=figsize)

if not use_mag:
    ax.set_yscale("log")
else:
    ax.invert_yaxis()
    ypl = -2.5*np.log10(ypl)

for algo_name in algo_vals:
    ymeans = np.array(df_algo_means.loc[algo_name]).reshape(-1)
    ystds = np.array(df_algo_stds.loc[algo_name]).reshape(-1)

    ybottom = ymeans-ystds
    ytop = ymeans+ystds

    ax.plot(xvals, ymeans, label=algo_name)
    ax.fill_between(xvals, ybottom, ytop, alpha=0.15)

if plot_pl:
    ax.plot(xpl, ypl, "gs", label="HD 1160 B", markersize=14)

txt = "fd"
ax.annotate(txt, (xvals[-1], np.array(df_algo_means.loc[txt])[-1,0]), color="C4", fontsize=16)
txt = "sng_pca"
ax.annotate(txt, (xvals[-1], np.array(df_algo_means.loc[txt])[-1,0]), color="C5", fontsize=16)

# ax.set_ylim(ylims)
ax.set_title("Contrast Curves grouped by Algorithm", fontsize=24)
ax.set_xlabel("Distance (arcsec)", fontsize=20)
ax.set_ylabel("$\Delta$mag" if use_mag else "5$\sigma$ Contrast", fontsize=20)
ax.tick_params(labelsize=14)
ax.legend(fontsize=16)
fig.tight_layout()
plt.show()

In [137]:
df_adi = df_master[["y", "t", "algo", "run"]].set_index("algo")
df_adi_grp = df_adi.loc["adi"].groupby("run")

s1 = np.array(df_adi_grp.apply(lambda r: r["y"]).loc["40-79_20"])
s2 = np.array(df_adi_grp.apply(lambda r: r["y"]).loc["45-74_20"])
s3 = np.array(df_adi_grp.apply(lambda r: r["y"]).loc["45-74_10"])

In [138]:
use_mag = False
plot_pl = True
# ylims = (0, 15) if use_mag else (1e-4, 2e-1)
# ylims = (1e-4, 1.0)
xpl = 21.403 * pxscale
ypl = 45.6 / 16231.
figsize = (10, 10)

plt.close()
fig, ax = plt.subplots(figsize=figsize)

if not use_mag:
    ax.set_yscale("log")


ax.plot(xvals, s1, label=run_name_map["40-79_20"])
ax.plot(xvals, s2, label=run_name_map["45-74_20"])
ax.plot(xvals, s3, label=run_name_map["45-74_10"])

if plot_pl:
    ax.plot(xpl, ypl, "gs", label="HD 1160 B", markersize=14)

ax.set_ylim(top=1.0)
ax.set_title("Contrast Curves for ADI grouped by Dataset", fontsize=24)
ax.set_xlabel("Distance (arcsec)", fontsize=20)
ax.set_ylabel("5$\sigma$ Contrast", fontsize=20)
ax.tick_params(labelsize=14)
ax.legend(fontsize=16)
fig.tight_layout()
plt.show()

In [139]:
df_samp = df_master[["y", "t", "algo", "run"]].set_index("run")
df_samp_grp = df_samp.loc["45-74_20"].groupby("algo")

s1 = np.array(df_samp_grp.apply(lambda r: r["y"]).loc["adi"])
s2 = np.array(df_samp_grp.apply(lambda r: r["y"]).loc["adi_ann"])
s3 = np.array(df_samp_grp.apply(lambda r: r["y"]).loc["asdi"])
s4 = np.array(df_samp_grp.apply(lambda r: r["y"]).loc["sdi"])
s5 = np.array(df_samp_grp.apply(lambda r: r["y"]).loc["npy_adi"])
s6 = np.array(df_samp_grp.apply(lambda r: r["y"]).loc["sng_pca"])
s7 = np.array(df_samp_grp.apply(lambda r: r["y"]).loc["dbl_pca"])
s8 = np.array(df_samp_grp.apply(lambda r: r["y"]).loc["ann_pca"])
s9 = np.array(df_samp_grp.apply(lambda r: r["y"]).loc["fd"])

In [140]:
use_mag = False
plot_pl = True
# ylims = (0, 15) if use_mag else (1e-4, 2e-1)
# ylims = (1e-4, 1.0)
xpl = 21.403 * pxscale
ypl = 45.6 / 16231.
figsize = (10, 10)

plt.close()
fig, ax = plt.subplots(figsize=figsize)

if not use_mag:
    ax.set_yscale("log")


ax.plot(xvals, s1, label="adi")
ax.plot(xvals, s2, label="adi_ann")
ax.plot(xvals, s3, label="asdi")
ax.plot(xvals, s4, label="sdi")
ax.plot(xvals, s5, label="npy_adi")
ax.plot(xvals, s6, label="sng_pca")
ax.plot(xvals, s7, label="dbl_pca")
ax.plot(xvals, s8, label="ann_pca")
ax.plot(xvals, s9, label="fd")

if plot_pl:
    ax.plot(xpl, ypl, "gs", label="HD 1160 B", markersize=14)

ax.set_ylim(top=1.0)
ax.set_title("Contrast Curves for 30ch 111fr grouped by Algorithm", fontsize=24)
ax.set_xlabel("Distance (arcsec)", fontsize=20)
ax.set_ylabel("5$\sigma$ Contrast", fontsize=20)
ax.tick_params(labelsize=14)
ax.legend(fontsize=16)
fig.tight_layout()
plt.show()